In [22]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [27]:
mnist_dataset, mnist_info = tfds.load(name="mnist", with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale (image,label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data= train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

# Model

In [40]:
input_size = 784
hidden_layer_size = 300
output_size = 10

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

# outline and loss function

In [41]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training

In [42]:
NUM_EPOCHS = 5

model.fit(train_data, epochs= NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 3s - loss: 0.2411 - accuracy: 0.9297 - val_loss: 0.1192 - val_accuracy: 0.9638 - 3s/epoch - 5ms/step
Epoch 2/5
540/540 - 2s - loss: 0.0911 - accuracy: 0.9714 - val_loss: 0.0922 - val_accuracy: 0.9720 - 2s/epoch - 4ms/step
Epoch 3/5
540/540 - 2s - loss: 0.0634 - accuracy: 0.9799 - val_loss: 0.0665 - val_accuracy: 0.9787 - 2s/epoch - 5ms/step
Epoch 4/5
540/540 - 2s - loss: 0.0486 - accuracy: 0.9841 - val_loss: 0.0590 - val_accuracy: 0.9830 - 2s/epoch - 4ms/step
Epoch 5/5
540/540 - 2s - loss: 0.0366 - accuracy: 0.9883 - val_loss: 0.0495 - val_accuracy: 0.9848 - 2s/epoch - 4ms/step


# Testing the Model

In [43]:
model.evaluate(test_data)

1/1 [==============================] - 1s 634ms/step - loss: 0.0734 - accuracy: 0.9782


[0.07335884124040604, 0.9782000184059143]